## To-do list
<li>Filter out large gap not alerting field</li>
<li>Find rolling average</li>
<li>Compare each new row with rolling average</li>

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.types import StructType
import json
emailSchema = StructType() \
        .add("metadata", StructType()\
             .add("path",StringType())\
             .add("_attachment_mimetype",StringType())\
             .add("type_prefix",StringType())\
             .add("host",StringType())\
             .add("json",StringType())\
             .add("producer",StringType())\
             .add("topic",StringType())\
             .add("_id",StringType())\
             .add("type",StringType())\
             .add("timestamp",LongType())\
            )\
        .add("data",StructType()\
             .add("code",StringType())\
             .add("system",StringType())\
             .add("uri_path",StringType())\
             .add("method",StringType())\
             .add("clientip",StringType())\
             .add("client",StringType())\
             .add("rec_date",StringType())\
             .add("dn",StringType())\
             .add("api",StringType())\
             .add("rec_timestamp",StringType())\
             .add("frontend",StringType())\
            )\

In [4]:

# Subscribe to 1 topic

# .option("kafka.bootstrap.servers", "188.185.79.229:9092")\
# .option("startingOffsets", "earliest") \

raw_data = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "monit-kafka.cern.ch:9092")\
.option("subscribe","cmsweb_logs")\
.load()\
.select(\
        from_json(col("value").cast("string"),emailSchema)\
            .getField("metadata").alias("metadata").getField("host").alias("host")\
        ,col("timestamp").alias("timestamp")\
        ,from_json(col("value").cast("string"),emailSchema)\
            .getField("data").alias("data").getField("system").alias("system")\
        ,from_json(col("value").cast("string"),emailSchema)\
            .getField("data").alias("data").getField("dn").alias("user")\
)

raw_data.printSchema()

root
 |-- host: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- system: string (nullable = true)
 |-- user: string (nullable = true)



In [62]:
groupped_data=raw_data\
.withWatermark("timestamp", "1 minute")\
.groupBy(window('timestamp', "1 minute", "1 minute"),"system")\
.agg(count("system").alias("count"))
groupped_data.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- system: string (nullable = true)
 |-- count: long (nullable = false)



In [49]:
raw_data_flow = raw_data.writeStream.queryName("email").outputMode("Append").format("memory").start()

In [63]:
groupped_data_flow = groupped_data.writeStream.queryName("groupped_email").outputMode("Append").format("memory").start()

First Error when implement rolling average
=============

In [13]:
from pyspark.sql.window import Window
w = Window.partitionBy("system").orderBy("window.start").rowsBetween(-1,1)
rolling_avg_data = groupped_data.withColumn("rolling_avg", avg("count")\
             .over(w))

In [25]:
rolling_avg_data_flow = rolling_avg_data.writeStream.queryName("rolling_avg").outputMode("Append").format("memory").start()

AnalysisException: 'Non-time-based windows are not supported on streaming DataFrames/Datasets;;\nWindow [avg(count#39L) windowspecdefinition(system#25, window#32-T60000ms.start ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, 1)) AS rolling_avg#50], [system#25], [window#32-T60000ms.start ASC NULLS FIRST]\n+- Project [window#32-T60000ms, system#25, count#39L, feature#44]\n   +- Project [window#32-T60000ms, system#25, count#39L, concat(system#25, cast(count#39L as string)) AS feature#44]\n      +- Aggregate [window#40-T60000ms, system#25], [window#40-T60000ms AS window#32-T60000ms, system#25, count(system#25) AS count#39L]\n         +- Filter isnotnull(timestamp#23-T60000ms)\n            +- Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as double) / cast(60000000 as double))) as double) = (cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongTy

Second error when implement multiple aggregation
=======================
Also same error after collect in HDFS

In [22]:
multi_group = groupped_data\
.withWatermark("window.start", "1 minute")\
.groupBy(window('window.start', "1 minute", "1 minute"),"system")\
.agg(avg("system").alias("avg"))\

Py4JJavaError: An error occurred while calling o201.withWatermark.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: makeCopy, tree:
'EventTimeWatermark 'window.start, interval 1 minutes
+- Aggregate [window#310-T60000ms, system#25], [window#310-T60000ms AS window#302-T60000ms, system#25, count(system#25) AS count#309L]
   +- Filter isnotnull(timestamp#23-T60000ms)
      +- Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as double) / cast(60000000 as double))) as double) = (cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as double) / cast(60000000 as double))) THEN (CEIL((cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as double) / cast(60000000 as double))) + cast(1 as bigint)) ELSE CEIL((cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as 

Third error: multiple aggregation
=======

In [26]:
avg_count = groupped_data\
.groupBy("system")\
.agg(avg("system").alias("avg"))\

In [27]:
avg_count_flow=avg_count.writeStream.queryName("avg_count").outputMode("Append").format("memory").start()

AnalysisException: 'Multiple streaming aggregations are not supported with streaming DataFrames/Datasets;;\nAggregate [system#25], [system#25, avg(cast(system#25 as double)) AS avg#3131]\n+- Aggregate [window#310-T60000ms, system#25], [window#310-T60000ms AS window#302-T60000ms, system#25, count(system#25) AS count#309L]\n   +- Filter isnotnull(timestamp#23-T60000ms)\n      +- Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as double) / cast(60000000 as double))) as double) = (cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as double) / cast(60000000 as double))) THEN (CEIL((cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as double) / cast(60000000 as double))) + cast(1 as bigint)) ELSE CEIL((cast((precisetimestampconversion(timestamp#23-T60000ms, TimestampType, LongType) - 0) as double) / cas

In [ ]:
concat_data = groupped_data.withColumn('feature', concat(col('system'), col('count')))
# Not working if include withcolumn: Result won't show

In [ ]:
accum = sc.accumulator(0)

In [ ]:
acc_data=groupped_data.writeStream.foreach(lambda s: accum.add(s)).queryName("acc").outputMode("Append").format("memory").start()
# acc_data=groupped_data.writeStream.foreach(lambda s: process_row(s)).queryName("acc").outputMode("Append").format("memory").start()

In [88]:
spark.streams.active

In [80]:
# groupped_data_flow.processAllAvailable()
# groupped_data_flow.lastProgress
# raw_data_flow.processAllAvailable()
# hdfs_data_flow.processAllAvailable()

In [ ]:
raw_data_flow.stop()

In [59]:
groupped_data_flow.stop()

In [30]:
alerts = spark.sql("select * from email")
alerts.show()

+----------------+--------------------+----------+--------------------+
|            host|           timestamp|    system|                user|
+----------------+--------------------+----------+--------------------+
|vocms055.cern.ch|2019-06-25 09:08:...|       dbs|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|    phedex|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|   reqmgr2|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|       dbs|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|   reqmgr2|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|    phedex|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|crabserver|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|crabserver|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|       dbs|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|crabserver|/DC=ch/DC=cern/OU...|
|vocms055.cern.ch|2019-06-25 09:08:...|crabserver|/DC=ch/DC=cern

In [82]:
alerts = spark.sql("select * from groupped_email")
alerts.show()

+--------------------+--------------------+-----+
|              window|              system|count|
+--------------------+--------------------+-----+
|[2019-06-25 09:27...|                 dbs| 1614|
|[2019-06-25 09:27...|    /wmarchive/data/|    6|
|[2019-06-25 09:27...|/couchdb/reqmgr_w...|   11|
|[2019-06-25 09:27...|/das/request?pid=...|    1|
|[2019-06-25 09:27...|/das/request?pid=...|    1|
|[2019-06-25 09:27...|                null|    0|
|[2019-06-25 09:27...|             reqmgr2| 4005|
|[2019-06-25 09:27...|      /server-status|   10|
|[2019-06-25 09:27...|/couchdb/wmstats/...|   25|
|[2019-06-25 09:27...|/das/request?view...|    1|
|[2019-06-25 09:27...|/das/request?inst...|    1|
|[2019-06-25 09:27...|       wmstatsserver|   38|
|[2019-06-25 09:27...|/couchdb/acdcserv...|   18|
|[2019-06-25 09:27...|/das/request?view...|    1|
|[2019-06-25 09:27...|/das/request?view...|    1|
|[2019-06-25 09:27...|           crabcache|   12|
|[2019-06-25 09:27...|          crabserver| 1673|


In order to fix this error, we have to change the 'checkpointLocation' even though it was once runnable
--

In [57]:
hdfs_data_flow.processAllAvailable()

StreamingQueryException: 'Job aborted.\n=== Streaming Query ===\nIdentifier: [id = 9f5c721d-220f-49d0-b532-e9e831a58652, runId = 18dbc10c-8e77-45f0-a135-4c122d80551e]\nCurrent Committed Offsets: {KafkaV2[Subscribe[cmsweb_logs]]: {"cmsweb_logs":{"17":86181591,"8":86181075,"11":86181588,"2":86181545,"5":86181051,"14":86180747,"13":86181589,"4":86180746,"16":86181589,"7":86181521,"1":86181587,"10":86181586,"19":86181358,"18":86181588,"9":86181085,"3":86181227,"12":86181595,"15":86181226,"6":86181300,"0":86181592}}}\nCurrent Available Offsets: {KafkaV2[Subscribe[cmsweb_logs]]: {"cmsweb_logs":{"17":86188146,"8":86187631,"11":86188139,"2":86188098,"5":86187605,"14":86187301,"13":86188144,"4":86187301,"16":86188133,"7":86188075,"1":86188143,"10":86188140,"19":86187912,"18":86188141,"9":86187641,"3":86187782,"12":86188147,"15":86187781,"6":86187836,"0":86188147}}}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nAggregate [window#4836-T60000ms, system#25], [window#4836-T60000

In [85]:
hdfs_data_flow=groupped_data.writeStream \
.outputMode("append")\
.format("parquet")\
 .option("path", "/cms/users/carizapo/ming/groupdata_cmsweb_logs") \
 .option("checkpointLocation", "/cms/users/carizapo/ming/checkpoint1_cmsweb_logs") \
 .outputMode("append") \
 .start()

In [ ]:
hdfs_data_flow.stop()